In [109]:
# bitget交易所 
# api key = bg_0761b22cd5153be02a13d736dc9a6ebd, 
# api secret = 258b6ea1d40990dc179b1a1cb4203df0221c5d11243e867c6be44918db69accb, 
# password = hrq1997SM

In [110]:
# 载入库
import ccxt
import arrow
import pyarrow
from typing import Generator, Literal, Optional
from datetime import datetime, timedelta
import json
import time
import pandas as pd
# 创建对象
exchange = ccxt.bitget({
    'apiKey': 'bg_0761b22cd5153be02a13d736dc9a6ebd',
    'secret': '258b6ea1d40990dc179b1a1cb4203df0221c5d11243e867c6be44918db69accb',
    'password':'hrq1997SM'
})
# withdrawal =  exchange.fetch_withdrawals(code="BTC")

In [111]:
# Exchange通用方法
def formatTrade(base: str, quote: str, trade: dict) -> dict:
    """格式化trade字段"""
    return {
        "datetime": trade["datetime"], # 交易时间
        "symbol": trade["symbol"], # 交易币种
        "id": trade["id"],  #交易所内部交易ID
        "side": trade["side"],  #buy/sell
        "base": base, #base symbol
        "quote": quote, #quote symbol
        "price": trade["price"],  #交易价格
        "amount": trade["amount"],  #交易数量
        "cost": trade["cost"],  #兑换数量
        "fees": trade["fees"],  #交易费信息 [{'symbol': 'USDT', 'amount': 1}]
    } # fees/fee？需要一致吗


def getHistoryTrade(
    func, pair: str, beginTs: int, endTs: int, limit: int
) -> list[dict]:
    """获取Trade数据"""
    result = {}

    beginTs = beginTs * 1000
    endTs = endTs * 1000 + 999
    base, quote = pair.split("/")
    queryTs = beginTs
    while queryTs <= endTs:
        trades = func(symbol=pair, since=queryTs, limit=limit)
        time.sleep(1)
        if not trades:
            break
        
        for trade in trades:
            if trade["timestamp"] > endTs:
                break

            if not trade["side"]:
                continue
                
            result[trade['id']] = formatTrade(base, quote, trade)
            
        queryTs = trades[-1]["timestamp"]
        
        if len(trades) < limit: #不到limit上限，说明已经获取完毕
            return result.values()
        
    return result.values()

def formatTransaction(item) -> dict:
    """格式化Deposit和Withdraw"""
    return {
        "id": item["id"],
        "txid": item["txid"],
        "network": item["network"],
        "datetime": item["datetime"],
        "addressFrom": item["addressFrom"],
        "addressTo": item["addressTo"],
        "currency": item["currency"],
        "amount": item["amount"],
        "fee": item["fee"] or {},
        "comment": item["comment"] or "",
    }

def getTransactions(func, code, beginTs: int, endTs: int, limit: int = 20) -> list[dict]:
    """获取一段小区间的Deposit或Withdrawal"""
    # print(beginTs, endTs)
    result = []
    
    beginTs = beginTs * 1000
    endTs = endTs * 1000 + 999
    queryTs = beginTs
    while queryTs <= endTs:
        #query_params = {"since": queryTs, "limit": limit}
        data = func(code = code, since = queryTs, limit = limit)
        time.sleep(1)
        if not data: #没有数据，不需要再查询了
            break

        formatData = []
        for item in data:
            if item["timestamp"] < beginTs:
                continue
            if item["timestamp"] > endTs:
                continue
            formatData.append(formatTransaction(item))

        if not formatData:
            break
        
        result.extend(formatData)
        queryTs = data[-1]["timestamp"] # 边界时间，limit没有获取完最后时间的全部数据
        if len(data) < limit:  #不到limit上限，说明不会出现边界时间
            queryTs += 1
        
    return result

def getHistoryTransaction(func, code, beginTs, endTs, days: int = 80, limit: int = 20):
    """获取Deposit或Withdrawal，有些交易所只能一次获取90天内的数据，需要切分区间获取"""
    history = {} #txid去重
    step = 3600 * 24 * days #时间参数不能大于80天，因为某些交易所(binance)不能获取跨度在90天以上的
    ts = beginTs
    while ts <= endTs:
        qEndTs = ts + step if ts + step < endTs else endTs #本次请求的结束时间
        result = getTransactions(func, code, ts, qEndTs, limit)
        if len(result) != 0:
            for tx in result:
                history[tx['id']] = tx  #去重
        ts = qEndTs + 1 #更新下次请求的开始时间
    return history.values()

In [112]:
# Bitget操作方法
import arrow
import pytz
import pandas as pd
from ccxt import AuthenticationError, BadSymbol, DDoSProtection

def syncCodeDeposit(exchange, code, beginTs, endTs, days = 80, limit = 100):
    data = getHistoryTransaction(
        exchange.fetch_deposits, 
        code,
        beginTs, 
        endTs, 
        limit,
    )
    print(f'共获取{code} deposit数据条数: ', len(data))
    return data

def syncDeposit(exchange, code, beginTs, endTs, days = 80, limit = 100):
    deposits = []
    try:
        if not code:
            deposits = syncDeposit(exchange, code, beginTs, endTs, limit = 100)
        else:
            for c in code:
                data = syncCodeDeposit(exchange, c, beginTs, endTs, limit = 100)
                deposits.extend(data)
        return pd.DataFrame.from_dict(deposits)

    except BadSymbol as e:
        return pd.DataFrame()
    except DDoSProtection as e:
        #raise RetryRequested(max_retries=3, seconds_to_wait=30) from e
        raise e
    except AuthenticationError as e:
        #elven_failed_notify(config.base_url, config.extra, message=str(e))
        raise e

def syncCodeWithdrawal(exchange, code, beginTs, endTs, days = 80, limit = 100):
    data = getHistoryTransaction(
        exchange.fetch_withdrawals, 
        code,
        beginTs, 
        endTs, 
        limit,
    )
    print(f'共获取{code} withdrawal数据条数: ', len(data))
    return data

def syncWithdrawal(exchange, code, beginTs, endTs, days = 80, limit=100):
    withdrawals = []
    try:
        if not code:
            withdrawals = syncWithdrawal(exchange, code, beginTs, endTs, limit = 100)
        else:
            for c in code:
                data = syncCodeWithdrawal(exchange, c, beginTs, endTs, limit = 100)
                withdrawals.extend(data)
        return pd.DataFrame.from_dict(withdrawals)

    except BadSymbol as e:
        return pd.DataFrame()
    except DDoSProtection as e:
        #raise RetryRequested(max_retries=3, seconds_to_wait=30) from e
        raise e
    except AuthenticationError as e:
        #elven_failed_notify(config.base_url, config.extra, message=str(e))
        raise e

def syncPairTrade(exchange, pair, beginTs, endTs, days = 95, limit=100):
    # 每次请求的时候最多获取100条数据
    data = getHistoryTrade(
        exchange.fetch_my_trades, 
        pair, 
        beginTs, 
        endTs,
        limit
    )
    print(f'共获取{pair}数据条数: ', len(data))
    return data
    
def syncTrade(exchange, pairs, beginTs, endTs, days = 95, limit=100):
    trades = []
    try:
        if not pairs:
            trades = syncTrade(exchange, pairs, beginTs, endTs, limit=100)
        else:
            for tradePair in pairs:
                data = syncPairTrade(exchange, tradePair, beginTs, endTs, limit=100)
                trades.extend(data)
        return pd.DataFrame.from_dict(trades)
        
    except BadSymbol as e:
        return pd.DataFrame()
    except DDoSProtection as e:
        #raise RetryRequested(max_retries=4, seconds_to_wait=21) from e
        raise e
    except AuthenticationError as e:
        #elven_failed_notify(config.base_url, config.extra, message=str(e))
        raise e

In [113]:
def processData(apiKey, secret, password, code, beginDate, endDate, timezone, pairs):
    """入口函数"""
    # 格式化时间为正确的时间戳
    tz = pytz.timezone(timezone) # 传入的是项目时间
    beginTime = datetime.strptime(beginDate + ' 00:00:00', '%Y-%m-%d %H:%M:%S')
    beginTs = int(tz.localize(beginTime).timestamp())

    endTime = datetime.strptime(endDate + ' 23:59:59', '%Y-%m-%d %H:%M:%S')
    endTs = int(tz.localize(endTime).timestamp())
    
    # bitget
    exchange = ccxt.bitget({
    'apiKey': apiKey,
    'secret': secret,
    'password': password
})

    # 生成结果数据

    # 获取数据有3个限制：1.每次最多获取90天 2. 每次最多获取100条 3. 需要给出币种code
    deposits = syncDeposit(exchange, code, beginTs, endTs, days=80, limit=100)
    withdrawals = syncWithdrawal(exchange, code, beginTs, endTs, days=80, limit=100)
    # trade每次最多获取100天，且需要传入交易对paris
    trades = syncTrade(exchange, pairs, beginTs, endTs, days = 95, limit=100)

    result = {
        'deposit': deposits,
        'withdrawal': withdrawals,
        'trade': trades,
    }

    return result

In [114]:
# 实际发起请求的传值
args = {
    "entityId": 122,
    "accountId": 438,
    "sourceId": 301,
    "timezone": "UTC",   #该数据源的时区, 0时区
    "entityTimezone": "Asia/Shanghai",  #项目的时区
    "beginDate": "",
    "endDate": "",
    "isMask": True,
    "apiKey": 'bg_0761b22cd5153be02a13d736dc9a6ebd',
    "apiSecret": '258b6ea1d40990dc179b1a1cb4203df0221c5d11243e867c6be44918db69accb',
    "password": 'hrq1997SM',
    "pairs": ["ETH/USDT", "BTC/USDT", "ETH/BTC", "BTC/USDC"],
    "code" : ["USDT", "ETH", "BTC", "USDC"]
}

In [115]:
beginDate = args['beginDate']
endDate = args['endDate']
if not beginDate:
    beginDate = '2023-12-12' # UTC+8
if not endDate: # UTC+8
    tz = pytz.timezone(args["entityTimezone"])
    endDate = (datetime.now(tz) - timedelta(days=1)).strftime('%Y-%m-%d')

args["pairs"] = args["pairs"] if "pairs" in args else ["ETH/USDT"]

result = processData(args["apiKey"], args["apiSecret"], args["password"], args["code"], beginDate, endDate, args["entityTimezone"], args["pairs"])

共获取USDT deposit数据条数:  3
共获取ETH deposit数据条数:  0
共获取BTC deposit数据条数:  0
共获取USDC deposit数据条数:  0
共获取USDT withdrawal数据条数:  0
共获取ETH withdrawal数据条数:  0
共获取BTC withdrawal数据条数:  0
共获取USDC withdrawal数据条数:  0
共获取ETH/USDT数据条数:  0
共获取BTC/USDT数据条数:  0
共获取ETH/BTC数据条数:  0
共获取BTC/USDC数据条数:  0


In [118]:
print(result)

{'deposit':                     id                                               txid  \
0  1143852942579867648  40ae9da0e733d2d29a5e70ef30dc4a7a9dd061e5ac140f...   
1  1144033246871420928  e7d24059e52ce4467bdd13fc0da502328cea26c45cbde7...   
2  1144054763470585856  f67fb0041b45fd6dfaa2aa7868329e8c97992c9bb1faea...   

      network                  datetime addressFrom  \
0  TRX(TRC20)  2024-02-20T15:08:33.948Z        None   
1  TRX(TRC20)  2024-02-21T03:05:01.841Z        None   
2  TRX(TRC20)  2024-02-21T04:30:31.798Z        None   

                            addressTo currency  amount fee comment  
0  TKN2tErwRoP9s5TSVLivm8xSmMcJqrVfYz     USDT    19.0  {}          
1  TKN2tErwRoP9s5TSVLivm8xSmMcJqrVfYz     USDT   424.0  {}          
2  TKN2tErwRoP9s5TSVLivm8xSmMcJqrVfYz     USDT   599.0  {}          , 'withdrawal': Empty DataFrame
Columns: []
Index: [], 'trade': Empty DataFrame
Columns: []
Index: []}


In [119]:
# 转换为DataFrame
dfs = {}
for key, value in result.items():
    dfs[key] = pd.DataFrame(value)

# 显示DataFrame
for key, df in dfs.items():
    print(key)
    print(df)
    print()

deposit
                    id                                               txid  \
0  1143852942579867648  40ae9da0e733d2d29a5e70ef30dc4a7a9dd061e5ac140f...   
1  1144033246871420928  e7d24059e52ce4467bdd13fc0da502328cea26c45cbde7...   
2  1144054763470585856  f67fb0041b45fd6dfaa2aa7868329e8c97992c9bb1faea...   

      network                  datetime addressFrom  \
0  TRX(TRC20)  2024-02-20T15:08:33.948Z        None   
1  TRX(TRC20)  2024-02-21T03:05:01.841Z        None   
2  TRX(TRC20)  2024-02-21T04:30:31.798Z        None   

                            addressTo currency  amount fee comment  
0  TKN2tErwRoP9s5TSVLivm8xSmMcJqrVfYz     USDT    19.0  {}          
1  TKN2tErwRoP9s5TSVLivm8xSmMcJqrVfYz     USDT   424.0  {}          
2  TKN2tErwRoP9s5TSVLivm8xSmMcJqrVfYz     USDT   599.0  {}          

withdrawal
Empty DataFrame
Columns: []
Index: []

trade
Empty DataFrame
Columns: []
Index: []



In [ ]:
exchange = ccxt.bitget({
    'apiKey': 'bg_0761b22cd5153be02a13d736dc9a6ebd',
    'secret': '258b6ea1d40990dc179b1a1cb4203df0221c5d11243e867c6be44918db69accb',
    'password':'hrq1997SM'
})
# 一点trade例子，withdraw和deposit先不看
# 指定交易对和时间范围
symbol = 'BTC/USDT'  # 交易对
beginDate = '2023-12-02'
tz = pytz.timezone(args['entityTimezone']) # 传入的是项目时间
beginTime = datetime.strptime(beginDate + ' 00:00:00', '%Y-%m-%d %H:%M:%S')
beginTs = int(tz.localize(beginTime).timestamp())
# since = int(1709258389000)

# 获取历史交易数据
# history_trades = exchange.fetch_my_trades(symbol, since=(beginTs*1000))
# withdrawal =  exchange.fetch_withdrawals(code="USDC", since=(beginTs*1000))
deposit = exchange.fetch_deposits(code='USDT')
print(deposit)